I imported libraries and pulled data.

In [54]:
import json
import pandas as pd
import requests
import networkx as nx
import gravis

In [32]:
with open(r"C:\Users\askin\Downloads\data.json", 'r', encoding='utf-8') as file:
    json_content = json.load(file)

This function below creates nodes for nx.graph with requested data type.

In [33]:
def node_maker(data, node_type):
    graph = nx.Graph()

    for m in data.get('nodes', []):
        if node_type in m and m[node_type]:
            node_value = m[node_type]
            graph.add_node(node_value)
    return graph

I wrote this function to add lines between nodes.

In [34]:
def edge_maker(data, graph, node_type, data_type_1, data_type_2):
    for m in data.get('nodes', []):
        if node_type in m and m[node_type]:
            wanted_value = m[node_type]
            data_values_1 = m.get(data_type_1, [])
            data_values_2 = m.get(data_type_2, [])

            for data_1 in data_values_1:
                if 0 < data_1 <= len(data['nodes']):
                    data_1_wanted = data['nodes'][data_1 - 1].get(node_type)
                    if data_1_wanted and data_1_wanted != wanted_value:
                        graph.add_edge(wanted_value, data_1_wanted)
                        
            for data_2 in data_values_2:
                if 0 < data_2 <= len(data['nodes']):
                    data_2_wanted = data['nodes'][data_2 - 1].get(node_type)
                    if data_2_wanted and data_2_wanted != wanted_value:
                        graph.add_edge(wanted_value, data_2_wanted)

    return graph


And here is the page rank algorithm.

In [35]:
def page_rank(graph, data_type, top_n=10):
    pagerank_values = nx.pagerank(graph)
    top_data_types = sorted(pagerank_values, key=pagerank_values.get, reverse=True)[:top_n]
    

    return top_data_types


And I called all the funcitons below there:

In [36]:
G_countries = node_maker(json_content, 'country')

In [37]:
G_countries = edge_maker(json_content, G_countries, 'country', 'advisors', 'student')

In [38]:
for u, v in G_countries.edges():
    G_countries[u][v]['weight'] = G_countries.number_of_edges(u, v)

Here is the best countries for Mathematics PhD:

In [39]:
page_rank_country = page_rank(G_countries, 'country', top_n = 10)
page_rank_country

['UnitedStates',
 'Germany',
 'UnitedKingdom',
 'France',
 'Canada',
 'Australia',
 'Italy',
 'Netherlands',
 'Switzerland',
 'Spain']

In [41]:
G_universities = node_maker(json_content, 'school')

In [42]:
G_universities = edge_maker(json_content, G_universities, 'school', 'advisors', 'student')

In [43]:
for u, v in G_universities.edges():
    weight = G_universities[u][v].get('weight', 0)
    G_universities[u][v]['weight'] = weight + 1

And the best Universities for Mathematics PhD

In [44]:
page_rank_school = page_rank(G_universities, 'school', top_n = 10)
page_rank_school

['Massachusetts Institute of Technology',
 'University of California, Berkeley',
 'Stanford University',
 'University of Wisconsin-Madison',
 'Harvard University',
 'University of Illinois at Urbana-Champaign',
 'University of Michigan',
 'University of California, Los Angeles',
 'Princeton University',
 'The University of Chicago']